## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
os.chdir('..')
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,68.0,87,39,5.99,scattered clouds
1,1,Padang,ID,-0.9492,100.3543,78.8,73,20,2.30,few clouds
2,2,Yulara,AU,-25.2406,130.9889,86.0,23,1,17.27,clear sky
3,3,Ponta Do Sol,PT,32.6667,-17.1000,54.0,67,12,1.99,few clouds
4,4,Busselton,AU,-33.6500,115.3333,61.0,96,100,3.00,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,68.00,87,39,5.99,scattered clouds
1,1,Padang,ID,-0.9492,100.3543,78.80,73,20,2.30,few clouds
2,2,Yulara,AU,-25.2406,130.9889,86.00,23,1,17.27,clear sky
5,5,Oliveira Dos Brejinhos,BR,-12.3169,-42.8961,70.95,91,100,2.80,moderate rain
8,8,Upington,ZA,-28.4478,21.2561,75.51,35,0,7.70,clear sky
11,11,Iquitos,PE,-3.7481,-73.2472,77.00,88,0,2.57,clear sky
14,14,Rikitea,PF,-23.1203,-134.9692,78.78,65,100,10.85,overcast clouds
24,24,Hermanus,ZA,-34.4187,19.2345,66.99,93,100,3.51,light rain
29,29,Christchurch,NZ,-43.5333,172.6333,68.00,60,20,20.71,few clouds
30,30,Atuona,PF,-9.8000,-139.0333,80.02,78,87,19.53,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                262
City                   262
Country                258
Lat                    262
Lng                    262
Max Temp               262
Humidity               262
Cloudiness             262
Wind Speed             262
Current Description    262
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                258
City                   258
Country                258
Lat                    258
Lng                    258
Max Temp               258
Humidity               258
Cloudiness             258
Wind Speed             258
Current Description    258
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,68.00,scattered clouds,-33.5906,26.8910,
1,Padang,ID,78.80,few clouds,-0.9492,100.3543,
2,Yulara,AU,86.00,clear sky,-25.2406,130.9889,
5,Oliveira Dos Brejinhos,BR,70.95,moderate rain,-12.3169,-42.8961,
8,Upington,ZA,75.51,clear sky,-28.4478,21.2561,
11,Iquitos,PE,77.00,clear sky,-3.7481,-73.2472,
14,Rikitea,PF,78.78,overcast clouds,-23.1203,-134.9692,
24,Hermanus,ZA,66.99,light rain,-34.4187,19.2345,
29,Christchurch,NZ,68.00,few clouds,-43.5333,172.6333,
30,Atuona,PF,80.02,overcast clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        
        print("Hotel not found... skipping.") 
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258 entries, 0 to 656
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 258 non-null    object 
 1   Country              258 non-null    object 
 2   Max Temp             258 non-null    float64
 3   Current Description  258 non-null    object 
 4   Lat                  258 non-null    float64
 5   Lng                  258 non-null    float64
 6   Hotel Name           245 non-null    object 
dtypes: float64(3), object(4)
memory usage: 26.1+ KB


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   245
Country                245
Max Temp               245
Current Description    245
Lat                    245
Lng                    245
Hotel Name             245
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))